In [67]:
!pip install pymongo

In [68]:
import pymongo
import pandas as pd
import numpy as np
import json

In [69]:
connessione = pymongo.MongoClient("mongodb://localhost:27017/")

In [70]:
print(connessione.list_database_names())

['WorldCup', 'admin', 'config', 'local']


# import dataset statistiche

In [71]:
connessione = pymongo.MongoClient("mongodb://localhost:27017/")
WorldCupDb = connessione["WorldCup"] #crea DB
WorldCupCollection = WorldCupDb["WorldCupStats"] #crea collezione all'interno del db

selection=WorldCupCollection.find()
dataset=[]
j=0
for i in selection:
    dataset.append(i)
    
df_stats=pd.DataFrame(dataset)
#dataset = dataset.rename({'nome_partita': 'partita'}, axis=1)
df_stats

,_id,tipo_di_partita,squadra_in_casa,squadra_in_trasferta,datetime_kick_off,gol_segnati_casa,gol_segnati_trasferta,rigori_finali_segnati_casa,rigori_finali_segnati_trasferta,nome_partita,...,gol_casa_1,gol_casa_2,gol_casa_3,gol_casa_4,gol_casa_5,gol_casa_6,gol_casa_7,cartellino_rosso_casa_1,gol_trasferta_3,gol_trasferta_4
0,63b835e872176b118ed982e8,Group A,QATAR,ECUADOR,2022-11-20 17:00:00,0,2,0,0,QATAR-ECUADOR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63b835e872176b118ed982e9,Group B,ENGLAND,IRAN,2022-11-21 14:00:00,6,2,0,0,ENGLAND-IRAN,...,Jude BELLINGHAM,Bukayo SAKA,Raheem STERLING,Bukayo SAKA,Marcus RASHFORD,Jack GREALISH,NaN,NaN,NaN,NaN
2,63b835e872176b118ed982ea,Group A,SENEGAL,NETHERLANDS,2022-11-21 17:00:00,0,2,0,0,SENEGAL-NETHERLANDS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63b835e872176b118ed982eb,Group B,UNITED STATES,WALES,2022-11-21 20:00:00,1,1,0,0,UNITED STATES-WALES,...,Tim WEAH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,63b835e972176b118ed982ec,Group C,ARGENTINA,SAUDI ARABIA,2022-11-22 11:00:00,1,2,0,0,ARGENTINA-SAUDI ARABIA,...,Lionel MESSI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,63b835fd72176b118ed98323,Quarter-final,ENGLAND,FRANCE,2022-12-10 20:00:00,1,2,0,0,ENGLAND-FRANCE,...,Harry KANE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,63b835fe72176b118ed98324,Semi-final,ARGENTINA,CROATIA,2022-12-13 20:00:00,3,0,0,0,ARGENTINA-CROATIA,...,Lionel MESSI,Julian ALVAREZ,Julian ALVAREZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,63b835fe72176b118ed98325,Semi-final,FRANCE,MOROCCO,2022-12-14 20:00:00,2,0,0,0,FRANCE-MOROCCO,...,Theo HERNANDEZ,Randal KOLO MUANI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,63b835ff72176b118ed98326,Play-off for third place,CROATIA,MOROCCO,2022-12-17 16:00:00,2,1,0,0,CROATIA-MOROCCO,...,Josko GVARDIOL,Mislav ORSIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# import dataset tweet

In [72]:
WorldCupCollection = WorldCupDb["WorldCupTweet"] #crea collezione all'interno del db
import pprint
pipeline=[{ "$group": {"_id":["$partita","$sentiment","$data_partita"], "sentiment_count": { "$count": {} }}}]
list10=(list(WorldCupCollection.aggregate( pipeline )))


In [73]:
df00_list= []
for i in range(len(list10)):
    tuple0 = tuple(list10[i]['_id'])
    multi_index = pd.MultiIndex.from_tuples([tuple0],
                                       names=['nome_partita',
                                              'sentiment',
                                              'kick_off_day'])
    value_count = list10[i]['sentiment_count']
    df00 = pd.DataFrame(value_count, 
                  columns=['tweet_count'],
                  index=multi_index
                 ).reset_index()[
                ['kick_off_day',
                 'nome_partita',
                 'sentiment',
                 'tweet_count']]
    df00_list.append(df00)
    
df01 = pd.concat(df00_list,
                 ignore_index=True)
df02 = df01.pivot_table(values='tweet_count'
                        ,index=['kick_off_day',
                               'nome_partita']
                        ,columns='sentiment'
                        ,aggfunc='mean'
                       ).reset_index()
df02['total_tweets'] = df02[['Negativo',
                             'Neutro',
                             'Positivo']].sum(axis=1)

In [74]:
df02.sort_values(by=['total_tweets'], inplace = True)
df02.head(5)

sentiment,kick_off_day,nome_partita,Negativo,Neutro,Positivo,total_tweets
3,2022-11-21,UNITED STATES-WALES,7,20,195,222
33,2022-11-29,IRAN-UNITED STATES,12,19,230,261
15,2022-11-24,URUGUAY-KOREA,70,121,80,271
49,2022-12-03,NETHERLANDS-UNITED STATES,4,5,285,294
16,2022-11-25,ENGLAND-UNITED STATES,7,17,290,314


# Import dataset ranking

In [75]:
WorldCupCollection = WorldCupDb["WorldCupRanking"] #crea collezione all'interno del db

selection=WorldCupCollection.find()
dataset=[]
for i in selection:
    dataset.append(i)
    
df_ranking=pd.DataFrame(dataset)
#dataset = dataset.rename({'nome_partita': 'partita'}, axis=1)
#df_ranking=df_ranking.drop(["_id"])
df_ranking

,_id,Rank pre,Rank post,Team,Previous Points,Total Points
0,63b7100f3154b7c81625fd8f,3,2,ARGENTINA,1770.65,1838.38
1,63b7100f3154b7c81625fd90,38,27,AUSTRALIA,1483.73,1533.97
2,63b7100f3154b7c81625fd91,2,4,BELGIUM,1821.92,1781.30
3,63b7100f3154b7c81625fd92,1,1,BRAZIL,1837.56,1840.77
4,63b7100f3154b7c81625fd93,43,33,CAMEROON,1484.95,1499.30
5,63b7100f3154b7c81625fd94,41,53,CANADA,1473.82,1431.64
6,63b7100f3154b7c81625fd95,31,32,COSTA RICA,1500.06,1500.18
7,63b7100f3154b7c81625fd96,12,7,CROATIA,1632.15,1727.62
8,63b7100f3154b7c81625fd97,10,18,DENMARK,1665.47,1608.11
9,63b7100f3154b7c81625fd98,44,41,ECUADOR,1463.74,1477.32


# Merging 

In [76]:
dataset_integrated=df_stats.merge(df02,on=["nome_partita","kick_off_day"])

In [77]:
dataset_integrated

,_id,tipo_di_partita,squadra_in_casa,squadra_in_trasferta,datetime_kick_off,gol_segnati_casa,gol_segnati_trasferta,rigori_finali_segnati_casa,rigori_finali_segnati_trasferta,nome_partita,...,gol_casa_5,gol_casa_6,gol_casa_7,cartellino_rosso_casa_1,gol_trasferta_3,gol_trasferta_4,Negativo,Neutro,Positivo,total_tweets
0,63b835e872176b118ed982e8,Group A,QATAR,ECUADOR,2022-11-20 17:00:00,0,2,0,0,QATAR-ECUADOR,...,NaN,NaN,NaN,NaN,NaN,NaN,249,383,368,1000
1,63b835e872176b118ed982e9,Group B,ENGLAND,IRAN,2022-11-21 14:00:00,6,2,0,0,ENGLAND-IRAN,...,Marcus RASHFORD,Jack GREALISH,NaN,NaN,NaN,NaN,204,401,395,1000
2,63b835e872176b118ed982ea,Group A,SENEGAL,NETHERLANDS,2022-11-21 17:00:00,0,2,0,0,SENEGAL-NETHERLANDS,...,NaN,NaN,NaN,NaN,NaN,NaN,250,364,386,1000
3,63b835e872176b118ed982eb,Group B,UNITED STATES,WALES,2022-11-21 20:00:00,1,1,0,0,UNITED STATES-WALES,...,NaN,NaN,NaN,NaN,NaN,NaN,7,20,195,222
4,63b835e972176b118ed982ec,Group C,ARGENTINA,SAUDI ARABIA,2022-11-22 11:00:00,1,2,0,0,ARGENTINA-SAUDI ARABIA,...,NaN,NaN,NaN,NaN,NaN,NaN,186,346,468,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,63b835fd72176b118ed98323,Quarter-final,ENGLAND,FRANCE,2022-12-10 20:00:00,1,2,0,0,ENGLAND-FRANCE,...,NaN,NaN,NaN,NaN,NaN,NaN,234,307,459,1000
60,63b835fe72176b118ed98324,Semi-final,ARGENTINA,CROATIA,2022-12-13 20:00:00,3,0,0,0,ARGENTINA-CROATIA,...,NaN,NaN,NaN,NaN,NaN,NaN,174,452,374,1000
61,63b835fe72176b118ed98325,Semi-final,FRANCE,MOROCCO,2022-12-14 20:00:00,2,0,0,0,FRANCE-MOROCCO,...,NaN,NaN,NaN,NaN,NaN,NaN,167,433,400,1000
62,63b835ff72176b118ed98326,Play-off for third place,CROATIA,MOROCCO,2022-12-17 16:00:00,2,1,0,0,CROATIA-MOROCCO,...,NaN,NaN,NaN,NaN,NaN,NaN,87,257,656,1000


In [80]:
dataset_integrated["Rank casa"]=0
dataset_integrated["Rank ospite"]=0
dataset_integrated["Punti Rank casa"]=0
dataset_integrated["Punti Rank ospite"]=0

for i in range(0,len(dataset_integrated)):
    for j in range (0,len(df_ranking)):
        if(dataset_integrated["squadra_in_casa"][i]==df_ranking["Team"][j]):
            dataset_integrated["Rank casa"][i]=df_ranking["Rank pre"][j]
           
            dataset_integrated["Punti Rank casa"][i]=df_ranking["Previous Points"][j]
        if(dataset_integrated["squadra_in_trasferta"][i]==df_ranking["Team"][j]):
            dataset_integrated["Rank ospite"][i]=df_ranking["Rank pre"][j]
            
            dataset_integrated["Punti Rank ospite"][i]=df_ranking["Previous Points"][j]

C:\Users\Utente\AppData\Local\Temp\ipykernel_17340\1639412089.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_integrated["Rank ospite"][i]=df_ranking["Rank pre"][j]
C:\Users\Utente\AppData\Local\Temp\ipykernel_17340\1639412089.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_integrated["Punti Rank ospite"][i]=df_ranking["Previous Points"][j]
C:\Users\Utente\AppData\Local\Temp\ipykernel_17340\1639412089.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [82]:
dataset_integrated=dataset_integrated.drop(["_id"],axis=1)

In [83]:
df_json=dataset_integrated.to_json(orient="records")
df_json = json.loads(df_json)
json.dumps(df_json, indent=4)
df_json[1]


{'tipo_di_partita': 'Group B',
 'squadra_in_casa': 'ENGLAND',
 'squadra_in_trasferta': 'IRAN',
 'datetime_kick_off': '2022-11-21 14:00:00',
 'gol_segnati_casa': 6,
 'gol_segnati_trasferta': 2,
 'rigori_finali_segnati_casa': 0,
 'rigori_finali_segnati_trasferta': 0,
 'nome_partita': 'ENGLAND-IRAN',
 'kick_off_day': '2022-11-21',
 'datetime_kick_off_string': '2022-11-21 14:00:00',
 'kick_off_day_string': '2022-11-21',
 'percentuale_possesso_palla_casa': 72,
 'percentuale_possesso_palla_trasferta': 19,
 'gol_totale_casa': 6,
 'gol_totale_trasferta': 2,
 'tiri_totale_casa': 13,
 'tiri_totale_trasferta': 8,
 'tiri_in_porta_casa': 7,
 'tiri_in_porta_trasferta': 3,
 'ammonizioni_casa': 0,
 'ammonizioni_trasferta': 2,
 'espulsioni_casa': 0,
 'espulsioni_trasferta': 0,
 'falli_subiti_casa': 9,
 'falli_subiti_trasferta': 14,
 'fuorigioco_casa': 2,
 'fuorigioco_trasferta': 2,
 'passaggi_casa': 809,
 'passaggi_trasferta': 224,
 'passaggi_completati_casa': 730,
 'passaggi_completati_trasferta': 154

In [85]:
WorldCupCollection = WorldCupDb["WorldCupDf"] #crea collezione all'interno del db
print(connessione.list_database_names())#si utilizza il db WorldCup

['WorldCup', 'admin', 'config', 'local']


In [87]:
for i in range(0,len(df_json)):
    inserimento = WorldCupCollection.insert_one(df_json[i])